In [ ]:
%%capture
!pip install ultralytics

In [ ]:
import os
import json
from PIL import Image
import numpy as np
import imutils
import cv2
import shutil
from skimage import io
from google.colab.patches import cv2_imshow
from skimage.measure import label, regionprops, find_contours
from ultralytics import YOLO
import torch
import torch.nn as nn

# **Preparing Datasets**

In [ ]:


""" Convert a mask to border image """
def mask_to_border(mask):
    h, w = mask.shape
    border = np.zeros((h, w))

    contours = find_contours(mask, 128)
    for contour in contours:
        for c in contour:
            x = int(c[0])
            y = int(c[1])
            border[x][y] = 255

    return border

""" Mask to bounding boxes """
def mask_to_bbox(mask):
    bboxes = []

    mask = mask_to_border(mask)
    lbl = label(mask)
    props = regionprops(lbl)
    for prop in props:
        x1 = prop.bbox[1]
        y1 = prop.bbox[0]

        x2 = prop.bbox[3]
        y2 = prop.bbox[2]

        bboxes.append([x1, y1, x2, y2])

    return bboxes




In [ ]:

if not os.path.exists('data'):
    for folder in ['images', 'labels']:
        for split in ['train', 'val','test']:
            os.makedirs(f'data/{folder}/{split}')

rm: cannot remove 'data': No such file or directory


**Kvasir-Seg**

In [ ]:
%%capture
!wget https://datasets.simula.no/downloads/kvasir-seg.zip
!unzip -x /content/kvasir-seg.zip

In [ ]:

f = open('/content/Kvasir-SEG/kavsir_bboxes.json')
json_file = json.load(f)

for i,name in enumerate(json_file.keys()):
  img_width=json_file[name]['width']
  img_height=json_file[name]['height']

  ans=""
  for bbox  in json_file[name]['bbox']:
    w=bbox['xmax']-bbox['xmin']
    h=bbox['ymax']-bbox['ymin']
    x_center=bbox['xmin']+w/2
    y_center=bbox['ymin']+h/2

    x_center=x_center/img_width
    y_center=y_center/img_height

    w=w/img_width
    h=h/img_height

    ans+=f'0 {x_center} {y_center} {w} {h}\n'


  if i<900:
    section="train"
  elif i<900+90:
    section="val"
  else:
    section="test"

  source_image_path = f'/content/Kvasir-SEG/images/{name}.jpg'
  shutil.copy(source_image_path, f'data/images/{section}/Kavir_{name}.jpg')

  with open(f'data/labels/{section}/Kavir_{name}.txt','w') as f:
    f.write(ans)




**CVC-ClinicDB**

In [ ]:
%%capture
!wget -O CVC-ClinicDB.rar https://dl.dropboxusercontent.com/scl/fi/ky766dwcxt9meq3aklkip/CVC-ClinicDB.rar?rlkey=61xclnrraadf1niqdvldlds93&e=3&st=8pojhfsi&dl=0
!unrar x CVC-ClinicDB.rar

In [ ]:
from skimage import io
from google.colab.patches import cv2_imshow
images=os.listdir("/content/CVC-ClinicDB/Original")
for i,name in enumerate(images):

  image=io.imread( f'/content/CVC-ClinicDB/Original/{name}')
  image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  mask = cv2.imread(f'/content/CVC-ClinicDB/Ground Truth/{name}')
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

  bboxes = mask_to_bbox(mask)
  ans=""
  for x,y,x2,y2 in bboxes:
    w=x2-x
    h=y2-y


    x=x+w/2
    y=y+h/2
    x=x/image.shape[1]
    y=y/image.shape[0]
    w=w/image.shape[1]
    h=h/image.shape[0]
    ans+=f'0 {x} {y} {w} {h}\n'

  if i<550:
    section="train"
  elif i<550+55:
    section="val"
  else:
    section="test"

  cv2.imwrite(f'data/images/{section}/CVC-ClinicDB_{name.replace("tif","jpg")}',image)
  with open(f'data/labels/{section}/CVC-ClinicDB_{name.replace(".tif",".txt")}','w') as f:
    f.write(ans)



**ETIS**

In [ ]:
%%capture
!wget -O ETIS-LaribPolypDB.rar https://dl.dropboxusercontent.com/scl/fi/7uvd98giierc0uq93d53x/ETIS-LaribPolypDB.rar?rlkey=qygrjpttbt9t3spt6z9cmvxew&st=0t7s9t2k&dl=0
!unrar x ETIS-LaribPolypDB.rar

In [ ]:

for i,name in enumerate(os.listdir("/content/ETIS-LaribPolypDB/ETIS-LaribPolypDB")):

  image=io.imread( f'/content/ETIS-LaribPolypDB/ETIS-LaribPolypDB/{name}')
  image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  mask = cv2.imread(f'/content/ETIS-LaribPolypDB/Ground Truth/p{name}')
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

  bboxes = mask_to_bbox(mask)
  ans=""
  for x,y,x2,y2 in bboxes:
    w=x2-x
    h=y2-y


    x=x+w/2
    y=y+h/2
    x=x/image.shape[1]
    y=y/image.shape[0]
    w=w/image.shape[1]
    h=h/image.shape[0]
    ans+=f'0 {x} {y} {w} {h}\n'

  if i<100:
    section="train"
  elif i<100+10:
    section="val"
  else:
    section="test"

  cv2.imwrite(f'data/images/{section}/ETIS_{name.replace("tif","jpg")}',image)
  with open(f'data/labels/{section}/ETIS_{name.replace(".tif",".txt")}','w') as f:
    f.write(ans)



**cvc-colondb**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("longvil/cvc-colondb")

print("Path to dataset files:", path)

100%|██████████| 91.3M/91.3M [00:01<00:00, 56.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/longvil/cvc-colondb/versions/1


In [ ]:

for i,name in enumerate(os.listdir(f'{path}/CVC-ColonDB/images')):

  image=io.imread( f'{path}/CVC-ColonDB/images/{name}')
  image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  mask = cv2.imread(f'{path}/CVC-ColonDB/masks/{name}')
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

  bboxes = mask_to_bbox(mask)
  ans=""
  for x,y,x2,y2 in bboxes:
    w=x2-x
    h=y2-y


    x=x+w/2
    y=y+h/2
    x=x/image.shape[1]
    y=y/image.shape[0]
    w=w/image.shape[1]
    h=h/image.shape[0]
    ans+=f'0 {x} {y} {w} {h}\n'
  if i<300:
    section="train"
  elif i<300+30:
    section="val"
  else:
    section="test"

  cv2.imwrite(f'data/images/{section}/CVC-ColonDB_{name.replace("tif","jpg")}',image)
  with open(f'data/labels/{section}/CVC-ColonDB_{name.replace(".tif",".txt")}','w') as f:
    f.write(ans)



# **Trainig**

In [ ]:
conf="train: /content/data/images/train\n\nval: /content/data/images/test\n\ntest: /content/data/images/test\n\nnc: 1\n\nnames: ['x']"
with open("VOC.yaml","w") as f:
  f.write(conf)

In [ ]:

class LightweightAttention(nn.Module):
    """Lightweight attention mechanism optimized for YOLO"""
    def __init__(self, channels, reduction_ratio=8):
        super().__init__()
        self.channels = channels
        reduced_channels = max(channels // reduction_ratio, 32)

        # Lightweight channel attention
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.channel_attention = nn.Sequential(
            nn.Linear(channels, reduced_channels),
            nn.SiLU(),
            nn.Linear(reduced_channels, channels),
            nn.Sigmoid()
        )

        # Lightweight spatial attention
        self.spatial_attention = nn.Sequential(
            nn.Conv2d(channels, 1, kernel_size=7, padding=3),
            nn.Sigmoid()
        )

        # Initialize weights
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Channel attention
        b, c, _, _ = x.shape
        channel_att = self.avg_pool(x).view(b, c)
        channel_att = self.channel_attention(channel_att).view(b, c, 1, 1)

        # Spatial attention
        spatial_att = self.spatial_attention(x)

        # Combine attentions
        return x * channel_att * spatial_att

class AttentionBlock(nn.Module):
    """Optimized attention block with residual connection"""
    def __init__(self, channels):
        super().__init__()
        self.attention = LightweightAttention(channels)
        self.norm = nn.BatchNorm2d(channels)
        self.act = nn.SiLU()

    def forward(self, x):
        identity = x
        x = self.attention(x)
        x = self.norm(x)
        x = self.act(x + identity)  # Residual connection
        return x

def add_attention_to_model(model):
    """Add optimized attention blocks strategically"""
    backbone = model.model.model

    # Add attention only at key feature extraction points
    attention_indices = [
        9,   # Middle features (reduced from previous version)
        12   # High-level features
    ]

    for idx in attention_indices:
        if idx < len(backbone):
            module = backbone[idx]
            if hasattr(module, 'conv'):
                channels = module.conv.out_channels
                attention = AttentionBlock(channels)
                backbone[idx] = nn.Sequential(module, attention)

    return model

def create_attention_model(pretrained_size='m'):
    """Create YOLO model with optimized attention"""
    model = YOLO(f'yolov8{pretrained_size}.pt')
    return add_attention_to_model(model)

def train_with_transfer_learning(
    pretrained_size='m',
    custom_dataset='/content/VOC.yaml',
    epochs=100,
    batch_size=16,
    imgsz=640,
    freeze_backbone=True
):
    """Train with optimized learning strategy"""
    model = create_attention_model(pretrained_size)

    # if freeze_backbone:
    #     # Freeze only early layers, allow more layers to train
    #     for param in model.model.model[:6].parameters():  # Reduced frozen layers
    #         param.requires_grad = False

    # Optimized training configuration
    train_args = {
        'data': custom_dataset,
        'epochs': epochs,
        'imgsz': imgsz,
        'batch': batch_size,
        'name': f'yolo_attention_{pretrained_size}',
        'patience': 30,  # Reduced patience for faster convergence
        'cos_lr': True,
        'warmup_epochs': 2.0,  # Shorter warmup
        'lr0': 0.01,  # Higher initial learning rate
        'lrf': 0.001,  # Lower final learning rate
        'momentum': 0.937,
        'weight_decay': 0.0005,
        'box': 7.5,  # Increased box loss weight
        'cls': 0.5,  # Reduced classification loss weight
        'dfl': 1.5,  # Adjusted DFL loss weight
        'close_mosaic': 10,  # Disable mosaic earlier
        'augment': True,
        'mixup': 0.1,  # Reduced mixup
        'copy_paste': 0.1  # Reduced copy-paste
    }

    results = model.train(**train_args)
    return model, results

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model, results=train_with_transfer_learning()

100%|██████████| 49.7M/49.7M [00:00<00:00, 304MB/s]


Ultralytics 8.3.75 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/VOC.yaml, epochs=100, time=None, patience=30, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo_attention_m, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes

100%|██████████| 755k/755k [00:00<00:00, 19.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 79.1MB/s]


...

    100/100      6.78G     0.4593     0.3195     0.9674          7        640: 100%|██████████| 116/116 [01:01<00:00,  1.90it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:02<00:00,  2.75it/s]

                   all        185        163       0.93      0.894       0.93      0.759



100 epochs completed in 1.840 hours.
Optimizer stripped from runs/detect/yolo_attention_m/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/yolo_attention_m/weights/best.pt, 52.0MB

Validating runs/detect/yolo_attention_m/weights/best.pt...
Ultralytics 8.3.75 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 218 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:06<00:00,  1.08s/it]


                   all        185        163      0.921       0.89      0.935      0.759
Speed: 0.3ms preprocess, 24.9ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to runs/detect/yolo_attention_m


**Saving the model**

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'
!cp -r /content/runs/detect "/content/drive/MyDrive/yolo poly8p"

# **Evaluation**

In [ ]:
predictions=model.val(data="VOC.yaml")

Ultralytics 8.3.75 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /content/data/labels/test.cache... 103 images, 50 backgrounds, 0 corrupt: 100%|██████████| 153/153 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


                   all        153        109      0.963      0.917      0.981      0.787
Speed: 3.0ms preprocess, 43.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/yolo_attention_m4
